<a href="https://colab.research.google.com/github/MDaniyalTariq/Urdu_Handwritten_Words_Recognition_through_CNN_Ensembling/blob/main/lstm_character_recognition_with_class_weights_and_lr_regulizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import os
import json
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import joblib

# Function to load and process the data
def load_data_from_directory(directory_path):
    data = []
    labels = []

    # Iterate over each file in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):
            file_path = os.path.join(directory_path, filename)

            # Load the data from the JSON file
            with open(file_path, "r") as f:
                file_data = json.load(f)

            # Extract sequences from the file
            sequences = []
            for character_variation in file_data:
                for i in range(0, len(character_variation), 5):  # Create sequences with 5 time steps
                    sequence = character_variation[i:i+5]
                    if len(sequence) == 5:
                        # Extract dx, dy, and timestamp from the dictionary and flatten into a list
                        flattened_sequence = [[point['dx'], point['dy'], point['timestamp']] for point in sequence]
                        sequences.append(flattened_sequence)

            # Add the sequences to the data list and label (filename without .json) to the labels list
            data.extend(sequences)
            labels.extend([filename.replace(".json", "")] * len(sequences))  # File name is the label

    return data, labels

# Preprocess the data (pad sequences and encode labels)
def preprocess_data(data, labels, sequence_length=50):
    # Pad or truncate sequences to make them of equal length
    sequences = pad_sequences(data, maxlen=sequence_length, dtype='float32', padding='post', truncating='post')

    # Encode the labels (character names)
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)
    labels_encoded = to_categorical(labels_encoded)  # One-hot encoding

    return np.array(sequences), np.array(labels_encoded), label_encoder

# Function to export the trained model
def save_model(model, model_path="trained_model.h5"):
    model.save(model_path)
    print(f"Model saved to {model_path}")

# Function to load the model
def load_trained_model(model_path="trained_model1.h5"):
    return load_model(model_path)

# Main function to load, preprocess, and train the model
def main():
    # Load the data from your directory
    directory_path = "/content/dataset/json"
    data, labels = load_data_from_directory(directory_path)

    # Preprocess the data
    sequences, labels_encoded, label_encoder = preprocess_data(data, labels)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(sequences, labels_encoded, test_size=0.2, random_state=42)

    # Compute class weights to handle class imbalance
    class_weights = compute_class_weight('balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1)) # Changed this line
    class_weight_dict = dict(zip(np.unique(np.argmax(y_train, axis=1)), class_weights)) # Changed this line

    # Example model (LSTM with increased complexity)
    model = Sequential([
        LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
        Dropout(0.3),
        LSTM(64),
        Dropout(0.3),
        Dense(len(label_encoder.classes_), activation="softmax")
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    # Add learning rate scheduler
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

    # Train the model with class weights, learning rate scheduler, and batch size adjustment
    model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), class_weight=class_weight_dict, callbacks=[lr_scheduler])

    # Save the trained model
    save_model(model)

    # Save label encoder
    joblib.dump(label_encoder, "label_encoder1.pkl")

# Function to predict a character from user input
def predict_character(user_input, model, label_encoder, sequence_length=50):
    """
    Predict the character based on the user input.
    :param user_input: List of dictionaries containing 'dx', 'dy', 'timestamp'.
    :param model: Trained model to use for predictions.
    :param label_encoder: Label encoder to decode predictions into characters.
    :param sequence_length: The sequence length for padding/truncating.
    :return: Predicted character label.
    """
    # Preprocess the user input
    flattened_input = [[point['dx'], point['dy'], point['timestamp']] for point in user_input]
    padded_input = pad_sequences([flattened_input], maxlen=sequence_length, dtype='float32', padding='post', truncating='post')

    # Make the prediction
    prediction = model.predict(padded_input)

    # Get the class index with the highest probability
    predicted_class_index = np.argmax(prediction)

    # Decode the class index to the actual character label
    predicted_character = label_encoder.inverse_transform([predicted_class_index])[0]

    return predicted_character

# Example of how to use the prediction function
if __name__ == "__main__":
    # Train and save the model first
    main()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_24 (LSTM)                       │ (None, 50, 128)             │          67,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 50, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_25 (LSTM)                       │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 35)                  │           2,275 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 119,267 (465.89 KB)

 Trainable params: 119,267 (465.89 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 177s 153ms/step - accuracy: 0.0441 - loss: 3.4780 - val_accuracy: 0.0607 - val_loss: 3.3874 - learning_rate: 0.0010
Epoch 2/10
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 169s 150ms/step - accuracy: 0.0563 - loss: 3.3631 - val_accuracy: 0.0620 - val_loss: 3.3933 - learning_rate: 0.0010
Epoch 3/10
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 181s 160ms/step - accuracy: 0.0655 - loss: 3.3140 - val_accuracy: 0.0783 - val_loss: 3.2596 - learning_rate: 0.0010
Epoch 4/10
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 191s 150ms/step - accuracy: 0.0774 - loss: 3.2383 - val_accuracy: 0.0940 - val_loss: 3.1790 - learning_rate: 0.0010
Epoch 5/10
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 200s 149ms/step - accuracy: 0.0814 - loss: 3.2088 - val_accuracy: 0.0837 - val_loss: 3.2098 - learning_rate: 0.0010
Epoch 6/10
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 203s 150ms/step - accuracy: 0.0908 - loss: 3.1737 - val_accuracy: 0.1007 - val_loss: 3.1383 - learning_rate: 0.0010
Epoch 7/10
1132/1132 ━━━━━━━━━━━━━━━━━━━━ 170s 150ms

Model saved to trained_model.h5


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'trained_model1.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [22]:
    import joblib
    model = load_trained_model("trained_model.h5")
    label_encoder = joblib.load("label_encoder1.pkl")

    # Sample user input (list of dictionaries with 'dx', 'dy', 'timestamp')
    user_input = [
        {"dx": 198.5, "dy": 97, "timestamp": 0},
        {"dx": 198.5, "dy": 97, "timestamp": 1},
        {"dx": 198.5, "dy": 97, "timestamp": 26},
        {"dx": 198.5, "dy": 97, "timestamp": 26},
        {"dx": 198.5, "dy": 97, "timestamp": 50},
    ]
    # Predict the character
    predicted_character = predict_character(user_input, model, label_encoder)

    # Print the predicted character
    print(f"Predicted Character: {predicted_character}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 502ms/step
Predicted Character: ل
